# RAG Retrieval Optimization - Query Transformation (Hyde) using Amazon Bedrock and Llamaindex

HyDE is a special type of query transformation technique that enhances the retrieval process and improve the relevance of retrieved documents. Instead of directly using the original query for retrieval, HyDE leverages a language model to generate a hypothetical document that captures the essence of the query's intent. This hypothetical document is then converted into an embedding, which is used to search for similar real documents in the knowledge base. The underlying concept is that the hypothetical document may be closer in the embedding space to relevant information than the original query itself, potentially leading to more accurate and contextually appropriate retrievals. HyDE has shown promising results in improving RAG performance, especially in zero-shot, and it has demonstrated effectiveness across various languages and tasks.


This example is built on referece llamaindex documentation available at - https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/query_transformations/HyDEQueryTransformDemo.ipynb

- Vector Database (Faiss / local)
- LLM (Amazon Bedrock - Claude3 Sonnet)
- Embeddings Model (Bedrock Titan Text Embedding v2.0)
- Datasets ( Amazons SEC-10k statments )

### > Setup

We start by importing necessary llamaindex libraries

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display

We select Anthropic Claude3 Sonnet as our LLM. For embedding model, we are selecting Amazon Titan Text Embed v2.0. Chunk size is set at 128 for this example.

In [ ]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v2:0")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
import nest_asyncio
nest_asyncio.apply()

We ingest and index the data stored in data directory. The amazon folder has SEC-10k files from 2022 and 2023.

### > Document Ingestion

In [ ]:
# load data
amazon_secfiles = SimpleDirectoryReader(input_dir="../data/lab03/amazon/").load_data()

# build index and query engine
index = VectorStoreIndex.from_documents(amazon_secfiles)

## Test query without HyDE

In [ ]:
query_str = "What were key challenges faced by Amazon in year 2022 and 2023?"

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(query_str)

### > Final response

In [ ]:
print(response)

## Test same query with HyDE enabled

In [ ]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)

### > Response with HyDE

In [ ]:
print(response)

### > Example hypothetical question generated by HYDE

In [ ]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [ ]:
print(hyde_doc)